In [2]:
import os
from google_drive_downloader import GoogleDriveDownloader as gdd

if not os.path.isdir('./omniglot_resized'):
    gdd.download_file_from_google_drive(file_id='1iaSFXIYC3AB8q9K_M-oVMa4pmB7yKMtI',
                                        dest_path='./omniglot_resized.zip',
                                        unzip=True)

assert os.path.isdir('./omniglot_resized')

Unzipping...Done.


In [154]:
""" Utility functions. """
import numpy as np
import os
import random
import tensorflow as tf

## Loss utilities
def cross_entropy_loss(pred, label, k_shot):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=tf.stop_gradient(label)) / k_shot)

def mse_loss(y_pred, y_true):
  return tf.keras.losses.MSE(y_true, y_pred)

def accuracy(labels, predictions):
  return tf.reduce_mean(tf.cast(tf.equal(labels, predictions), dtype=tf.float32))
    

In [155]:
"""Data loading scripts"""
import numpy as np
import os
import random
import tensorflow as tf
from scipy import misc
import imageio

def get_images(paths, labels, n_samples=None, shuffle=True):
  """
  Takes a set of character folders and labels and returns paths to image files
  paired with labels.
  Args:
    paths: A list of character folders
    labels: List or numpy array of same length as paths
    n_samples: Number of images to retrieve per character
  Returns:
    List of (label, image_path) tuples
  """
  if n_samples is not None:
    sampler = lambda x: random.sample(x, n_samples)
  else:
    sampler = lambda x: x
  images_labels = [(i, os.path.join(path, image))
           for i, path in zip(labels, paths)
           for image in sampler(os.listdir(path))]
  if shuffle:
    random.shuffle(images_labels)
  return images_labels


def image_file_to_array(filename, dim_input):
  """
  Takes an image path and returns numpy array
  Args:
    filename: Image filename
    dim_input: Flattened shape of image
  Returns:
    1 channel image
  """
  image = imageio.imread(filename)
  image = image.reshape([dim_input])
  image = image.astype(np.float32) / 255.0
  image = 1.0 - image
  return image


class DataGenerator(object):
  """
  Data Generator capable of generating batches of Omniglot data.
  A "class" is considered a class of omniglot digits.
  """

  def __init__(self, num_classes, num_samples_per_class, num_meta_test_classes, num_meta_test_samples_per_class, config={}):
    """
    Args:
      num_classes: Number of classes for classification (K-way)
      num_samples_per_class: num samples to generate per class in one batch
      num_meta_test_classes: Number of classes for classification (K-way) at meta-test time
      num_meta_test_samples_per_class: num samples to generate per class in one batch at meta-test time
      batch_size: size of meta batch size (e.g. number of functions)
    """
    self.num_samples_per_class = num_samples_per_class
    self.num_classes = num_classes
    self.num_meta_test_samples_per_class = num_meta_test_samples_per_class
    self.num_meta_test_classes = num_meta_test_classes

    data_folder = config.get('data_folder', './omniglot_resized')
    self.img_size = config.get('img_size', (28, 28))

    self.dim_input = np.prod(self.img_size)
    # self.dim_output = self.num_classes
    self.dim_output = np.prod(self.img_size)

    character_folders = [os.path.join(data_folder, family, character)
               for family in os.listdir(data_folder)
               if os.path.isdir(os.path.join(data_folder, family))
               for character in os.listdir(os.path.join(data_folder, family))
               if os.path.isdir(os.path.join(data_folder, family, character))]

    random.seed(123)
    random.shuffle(character_folders)
    num_val = 100
    num_train = 1100
    self.metatrain_character_folders = character_folders[: num_train]
    self.metaval_character_folders = character_folders[
      num_train:num_train + num_val]
    self.metatest_character_folders = character_folders[
      num_train + num_val:]

  def sample_batch(self, batch_type, batch_size, shuffle=True, swap=False):
    """
    Samples a batch for training, validation, or testing
    Args:
      batch_type: meta_train/meta_val/meta_test
      shuffle: randomly shuffle classes or not
      swap: swap number of classes (N) and number of samples per class (K) or not
    Returns:
      A a tuple of (1) Image batch and (2) Label batch where
      image batch has shape [B, N, K, 784] and label batch has shape [B, N, K, N] if swap is False
      where B is batch size, K is number of samples per class, N is number of classes
    """
    if batch_type == "meta_train":
      folders = self.metatrain_character_folders
      num_classes = self.num_classes
      num_samples_per_class = self.num_samples_per_class
    elif batch_type == "meta_val":
      folders = self.metaval_character_folders
      num_classes = self.num_classes
      num_samples_per_class = self.num_samples_per_class
    else:
      folders = self.metatest_character_folders
      num_classes = self.num_meta_test_classes
      num_samples_per_class = self.num_meta_test_samples_per_class
    all_image_batches, all_label_batches = [], []
    for i in range(batch_size):
      sampled_character_folders = random.sample(
        folders, num_classes)
      labels_and_images = get_images(sampled_character_folders, range(
        num_classes), n_samples=num_samples_per_class, shuffle=False)
      labels = [li[0] for li in labels_and_images]
      images = [image_file_to_array(
        li[1], self.dim_input) for li in labels_and_images]
      images = np.stack(images)
      labels = np.array(labels).astype(np.int32)
      labels = np.reshape(
        labels, (num_classes, num_samples_per_class))
      labels = np.eye(num_classes, dtype=np.float32)[labels]
      images = np.reshape(
        images, (num_classes, num_samples_per_class, -1))

      batch = np.concatenate([labels, images], 2)
      if shuffle:
        for p in range(num_samples_per_class):
          np.random.shuffle(batch[:, p])

      # labels = batch[:, :, :num_classes]
      images = batch[:, :, num_classes:]

      if swap:
        # labels = np.swapaxes(labels, 0, 1)
        images = np.swapaxes(images, 0, 1)

      all_image_batches.append(images)
      # all_label_batches.append(labels)
    all_image_batches = np.stack(all_image_batches)
    # all_label_batches = np.stack(all_label_batches)
    # return all_image_batches, all_label_batches
    return all_image_batches, all_image_batches

In [199]:
"""Convolutional layers used by MAML model."""
seed = 123

class GeneratorNetwork(tf.keras.layers.Layer):
  def __init__(self, channels, dim_hidden, dim_output, img_size):
    super(GeneratorNetwork, self).__init__()
    self.channels = channels
    self.dim_hidden = dim_hidden
    self.dim_output = dim_output
    self.img_size = img_size

    weights = {}

    dtype = tf.float32
    weight_initializer =  tf.keras.initializers.GlorotUniform()

    weights['G_W1'] = tf.Variable(weight_initializer(shape=[784 * self.channels, self.dim_hidden]), name='G_W1', dtype=dtype)  
    weights['G_b1'] = tf.Variable(tf.zeros([self.dim_hidden]), name='G_b1')
    
    weights['G_W2'] = tf.Variable(weight_initializer(shape=[self.dim_hidden, self.dim_hidden]), name='G_W2', dtype=dtype)  
    weights['G_b2'] = tf.Variable(tf.zeros([self.dim_hidden]), name='G_b2')

    weights['G_W3'] = tf.Variable(weight_initializer(shape=[self.dim_hidden, self.dim_output]), name='G_W3', dtype=dtype)  
    weights['G_b3'] = tf.Variable(tf.zeros([self.dim_output]), name='G_b3')

    self.net_weights = weights

  def call(self, inp, weights):
    # Concatenate Mask and Data
    # x, m = inp
    # inputs = tf.concat(values = inp, axis = 1) 

    G_h1 = tf.nn.relu(tf.matmul(inp, weights['G_W1']) + weights['G_b1'])
    G_h2 = tf.nn.relu(tf.matmul(G_h1, weights['G_W2']) + weights['G_b2'])   
    # MinMax normalized output
    G_prob = tf.nn.sigmoid(tf.matmul(G_h2, weights['G_W3']) + weights['G_b3']) 
    return G_prob

In [197]:
"""Convolutional layers used by MAML model."""
seed = 123

class DiscriminatorNetwork(tf.keras.layers.Layer):
  def __init__(self, channels, dim_hidden, dim_output, img_size):
    super(DiscriminatorNetwork, self).__init__()
    self.channels = channels
    self.dim_hidden = dim_hidden
    self.dim_output = dim_output
    self.img_size = img_size

    weights = {}

    dtype = tf.float32
    weight_initializer =  tf.keras.initializers.GlorotUniform()

    weights['D_W1'] = tf.Variable(weight_initializer(shape=[784, self.dim_hidden]), name='D_W1', dtype=dtype) # Data + Hint as inputs
    weights['D_b1'] = tf.Variable(tf.zeros([self.dim_hidden]), name='D_b1')
  
    weights['D_W2'] = tf.Variable(weight_initializer(shape=[self.dim_hidden, self.dim_hidden]), name='D_W2', dtype=dtype)
    weights['D_b2'] = tf.Variable(tf.zeros([self.dim_hidden]), name='D_b2')
  
    weights['D_W3'] = tf.Variable(weight_initializer(shape=[self.dim_hidden, self.dim_input]), name='D_W3', dtype=dtype)
    weights['D_b3'] = tf.Variable(tf.zeros([self.dim_output]), name='D_b3')

    self.net_weights = weights

  def call(self, inp, weights):
    # inputs = tf.concat(values = [x, h], axis = 1) 
    D_h1 = tf.nn.relu(tf.matmul(inp, weights['D_W1']) + weights['D_b1'])  
    D_h2 = tf.nn.relu(tf.matmul(D_h1, weights['D_W2']) + weights['D_b2'])
    D_logit = tf.matmul(D_h2, weights['D_W3']) + weights['D_b3']
    D_prob = tf.nn.sigmoid(D_logit)
    return D_prob

In [203]:
"""MAML model code"""
import numpy as np
import sys
import tensorflow as tf
from functools import partial


class MAML(tf.keras.Model):
  def __init__(self, dim_input=1, dim_output=1,
               num_inner_updates=1,
               inner_update_lr=0.4, dim_hidden=300, k_shot=5, learn_inner_update_lr=False, network_type=GeneratorNetwork):
    super(MAML, self).__init__()
    self.dim_input = dim_input
    self.dim_output = dim_output
    self.inner_update_lr = inner_update_lr
    self.loss_func = mse_loss
    self.dim_hidden = dim_hidden
    self.channels = 1
    self.img_size = int(np.sqrt(self.dim_input/self.channels))

    # outputs_ts[i] and losses_ts_post[i] are the output and loss after i+1 inner gradient updates
    losses_tr_pre, outputs_tr, losses_ts_post, outputs_ts = [], [], [], []
    # accuracies_tr_pre, accuracies_ts = [], []

    # for each loop in the inner training loop
    outputs_ts = [[]]*num_inner_updates
    losses_ts_post = [[]]*num_inner_updates
    # accuracies_ts = [[]]*num_inner_updates

    # Define the weights - these should NOT be directly modified by the
    # inner training loop
    tf.random.set_seed(seed)
    self.gen_network_layers = network_type(self.channels, self.dim_hidden, self.dim_output, self.img_size)

    self.learn_inner_update_lr = learn_inner_update_lr
    if self.learn_inner_update_lr:
      self.inner_update_lr_dict = {}
      for key in self.gen_network_layers.net_weights.keys():
        self.inner_update_lr_dict[key] = [tf.Variable(self.inner_update_lr, name='inner_update_lr_%s_%d' % (key, j)) for j in range(num_inner_updates)]
  

  def call(self, inp, meta_batch_size=25, num_inner_updates=1, other_network):
    def task_inner_loop(inp, reuse=True,
                      meta_batch_size=25, num_inner_updates=1):
      """
        Perform gradient descent for one task in the meta-batch (i.e. inner-loop).
        Args:
          inp: a tuple (input_tr, input_ts, label_tr, label_ts), where input_tr and label_tr are the inputs and
            labels used for calculating inner loop gradients and input_ts and label_ts are the inputs and
            labels used for evaluating the model after inner updates.
            Should be shapes:
              input_tr: [N*K, 784]
              input_ts: [N*K, 784]
              label_tr: [N*K, N]
              label_ts: [N*K, N]
        Returns:
          task_output: a list of outputs, losses and accuracies at each inner update
      """
      # the inner and outer loop data
      input_tr, input_ts, label_tr, label_ts = inp

      # weights corresponds to the initial weights in MAML (i.e. the meta-parameters)
      weights = self.gen_network_layers.net_weights

      # the predicted outputs, loss values, and accuracy for the pre-update model (with the initial weights)
      # evaluated on the inner loop training data
      task_output_tr_pre, task_loss_tr_pre = None, None
      # task_output_tr_pre, task_loss_tr_pre, task_accuracy_tr_pre = None, None, None

      # lists to keep track of outputs, losses, and accuracies of test data for each inner_update
      # where task_outputs_ts[i], task_losses_ts[i], task_accuracies_ts[i] are the output, loss, and accuracy
      # after i+1 inner gradient updates
      task_outputs_ts, task_losses_ts = [], []
      # task_outputs_ts, task_losses_ts, task_accuracies_ts = [], [], []
  
      # perform num_inner_updates to get modified weights
      # modified weights should be used to evaluate performance
      # Note that at each inner update, always use input_tr and label_tr for calculating gradients
      # and use input_ts and labels for evaluating performance

      # https://www.tensorflow.org/guide/advanced_autodiff#higher-order_gradients
      # https://www.tensorflow.org/api_docs/python/tf/GradientTape
      with tf.GradientTape() as inner_tape:
        # TRAIN 0
        task_output_tr_pre = self.gen_network_layers(input_tr, weights)
        task_loss_tr_pre = self.loss_func(task_output_tr_pre, label_tr)
        grads = inner_tape.gradient(task_loss_tr_pre, list(weights.values()))
        gradients = dict(zip(weights.keys(), grads))
        updated_weights = dict(zip(weights.keys(), [weights[key] - self.learn_inner_update_lr*gradients[key] for key in weights.keys()]))

        # EVAL WITH TEST 0
        task_outputs_ts.append(self.gen_network_layers(input_ts, updated_weights))
        task_losses_ts.append(self.loss_func(task_outputs_ts[0], label_ts))
        
        for i in range(num_inner_updates-1):
          # TRAIN i
          task_output_tr = self.gen_network_layers(input_tr, updated_weights)
          task_loss_tr = self.loss_func(task_output_tr, label_tr)
          grads = inner_tape.gradient(task_loss_tr, list(updated_weights.values()))
          gradients = dict(zip(updated_weights.keys(), grads))
          updated_weights = dict(zip(updated_weights.keys(), [updated_weights[key] - self.learn_inner_update_lr*gradients[key] for key in updated_weights.keys()]))
        
          # EVAL WITH TEST i
          task_outputs_ts.append(self.gen_network_layers(input_ts, updated_weights))
          task_losses_ts.append(self.loss_func(task_outputs_ts[i], label_ts))
          
      #############################

      # Compute accuracies from output predictions
      # task_accuracy_tr_pre = accuracy(tf.argmax(input=label_tr, axis=1), tf.argmax(input=tf.nn.softmax(task_output_tr_pre), axis=1))

      # for j in range(num_inner_updates):
      #   task_accuracies_ts.append(accuracy(tf.argmax(input=label_ts, axis=1), tf.argmax(input=tf.nn.softmax(task_outputs_ts[j]), axis=1)))

      # task_output = [task_output_tr_pre, task_outputs_ts, task_loss_tr_pre, task_losses_ts, task_accuracy_tr_pre, task_accuracies_ts]
      task_output = [task_output_tr_pre, task_outputs_ts, task_loss_tr_pre, task_losses_ts]

      return task_output

    input_tr, input_ts, label_tr, label_ts = inp
    # to initialize the batch norm vars, might want to combine this, and not run idx 0 twice.
    unused = task_inner_loop((input_tr[0], input_ts[0], label_tr[0], label_ts[0]),
                          False,
                          meta_batch_size,
                          num_inner_updates)
    out_dtype = [tf.float32, [tf.float32]*num_inner_updates, tf.float32, [tf.float32]*num_inner_updates]
    # out_dtype.extend([tf.float32, [tf.float32]*num_inner_updates])
    task_inner_loop_partial = partial(task_inner_loop, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)
    result = tf.map_fn(task_inner_loop_partial,
                    elems=(input_tr, input_ts, label_tr, label_ts),
                    dtype=out_dtype,
                    parallel_iterations=meta_batch_size)
    return result
   

In [201]:
"""Model training code"""
"""
Usage Instructions:
  5-way, 1-shot omniglot:
    python main.py --meta_train_iterations=15000 --meta_batch_size=25 --k_shot=1 --inner_update_lr=0.4 --num_inner_updates=1 --logdir=logs/omniglot5way/
  20-way, 1-shot omniglot:
    python main.py --meta_train_iterations=15000 --meta_batch_size=16 --k_shot=1 --n_way=20 --inner_update_lr=0.1 --num_inner_updates=5 --logdir=logs/omniglot20way/
  To run evaluation, use the '--meta_train=False' flag and the '--meta_test_set=True' flag to use the meta-test set.
"""
import csv
import numpy as np
import pickle
import random
import tensorflow as tf

def outer_train_step(inp, model, optim, meta_batch_size=25, num_inner_updates=1):
  with tf.GradientTape(persistent=False) as outer_tape:
    result = model(inp, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)

    # outputs_tr, outputs_ts, losses_tr_pre, losses_ts, accuracies_tr_pre, accuracies_ts = result
    outputs_tr, outputs_ts, losses_tr_pre, losses_ts = result

    total_losses_ts = [tf.reduce_mean(loss_ts) for loss_ts in losses_ts]

  gradients = outer_tape.gradient(total_losses_ts[-1], model.trainable_variables)
  optim.apply_gradients(zip(gradients, model.trainable_variables))

  total_loss_tr_pre = tf.reduce_mean(losses_tr_pre)
  # total_accuracy_tr_pre = tf.reduce_mean(accuracies_tr_pre)
  # total_accuracies_ts = [tf.reduce_mean(accuracy_ts) for accuracy_ts in accuracies_ts]

  return outputs_tr, outputs_ts, total_loss_tr_pre, total_losses_ts#, total_accuracy_tr_pre, total_accuracies_ts

def outer_eval_step(inp, model, meta_batch_size=25, num_inner_updates=1):
  result = model(inp, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)

  # outputs_tr, outputs_ts, losses_tr_pre, losses_ts, accuracies_tr_pre, accuracies_ts = result
  outputs_tr, outputs_ts, losses_tr_pre, losses_ts = result

  total_loss_tr_pre = tf.reduce_mean(losses_tr_pre)
  total_losses_ts = [tf.reduce_mean(loss_ts) for loss_ts in losses_ts]

  # total_accuracy_tr_pre = tf.reduce_mean(accuracies_tr_pre)
  # total_accuracies_ts = [tf.reduce_mean(accuracy_ts) for accuracy_ts in accuracies_ts]

  return outputs_tr, outputs_ts, total_loss_tr_pre, total_losses_ts#, total_accuracy_tr_pre, total_accuracies_ts  


def meta_train_fn(model, exp_string, data_generator,
               n_way=5, meta_train_iterations=15000, meta_batch_size=25,
               log=True, logdir='/tmp/data', k_shot=1, num_inner_updates=1, meta_lr=0.001):
  SUMMARY_INTERVAL = 10
  SAVE_INTERVAL = 100
  PRINT_INTERVAL = 10  
  TEST_PRINT_INTERVAL = PRINT_INTERVAL*5

  # pre_accuracies, post_accuracies = [], []
  pre_losses, post_losses = [], []

  num_classes = data_generator.num_classes

  optimizer = tf.keras.optimizers.Adam(learning_rate=meta_lr)

  for itr in range(meta_train_iterations):
    # sample a batch of training data and partition into
    # the support/training set (input_tr, label_tr) and the query/test set (input_ts, label_ts)
    # NOTE: The code assumes that the support and query sets have the same number of examples.
    batch = data_generator.sample_batch("meta_train", meta_batch_size, shuffle=False, swap=False)
    #batch[0].shape(images) = (meta_batch_size, num_classes, k_shot*2, 784)
    #batch[1].shape(labels) = (meta_batch_size, num_classes, k_shot*2, num_classes)
    input_tr, input_ts = tf.split(batch[0], num_or_size_splits=2, axis=2)
    input_tr = tf.reshape(input_tr, [meta_batch_size, num_classes*k_shot, 784])
    input_ts = tf.reshape(input_ts, [meta_batch_size, num_classes*k_shot, 784])
    label_tr, label_ts = tf.split(batch[1], num_or_size_splits=2, axis=2)
    label_tr = tf.reshape(label_tr, [meta_batch_size, num_classes*k_shot, 784])
    label_ts = tf.reshape(label_ts, [meta_batch_size, num_classes*k_shot, 784])

    inp = (input_tr, input_ts, label_tr, label_ts)
    
    result = outer_train_step(inp, model, optimizer, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)

    if itr % SUMMARY_INTERVAL == 0:
      # pre_accuracies.append(result[-2])
      # post_accuracies.append(result[-1][-1])
      pre_losses.append(result[2])
      post_losses.append(result[3][-1])

    if (itr!=0) and itr % PRINT_INTERVAL == 0:
      # print_str = 'Iteration %d: pre-inner-loop train accuracy: %.5f, post-inner-loop test accuracy: %.5f' % (itr, np.mean(pre_accuracies), np.mean(post_accuracies))
      # print(print_str)
      # pre_accuracies, post_accuracies = [], []
      print_str = 'Iteration %d: pre-inner-loop train loss: %.5f, post-inner-loop test loss: %.5f' % (itr, np.mean(pre_losses), np.mean(post_losses))
      print(print_str)
      pre_losses, post_losses = [], []

    if (itr!=0) and itr % TEST_PRINT_INTERVAL == 0:
      # sample a batch of validation data and partition it into
      # the support/training set (input_tr, label_tr) and the query/test set (input_ts, label_ts)
      # NOTE: The code assumes that the support and query sets have the same number of examples.
      batch = data_generator.sample_batch("meta_val", meta_batch_size, shuffle=False, swap=False)
      #batch[0].shape(images) = (meta_batch_size, num_classes, k_shot*2, 784)
      #batch[1].shape(labels) = (meta_batch_size, num_classes, k_shot*2, num_classes)
      input_tr, input_ts = tf.split(batch[0], num_or_size_splits=2, axis=2)
      input_tr = tf.reshape(input_tr, [meta_batch_size, num_classes*k_shot, 784])
      input_ts = tf.reshape(input_ts, [meta_batch_size, num_classes*k_shot, 784])
      label_tr, label_ts = tf.split(batch[1], num_or_size_splits=2, axis=2)
      label_tr = tf.reshape(label_tr, [meta_batch_size, num_classes*k_shot, 784])
      label_ts = tf.reshape(label_ts, [meta_batch_size, num_classes*k_shot, 784])

      inp = (input_tr, input_ts, label_tr, label_ts)
      result = outer_eval_step(inp, model, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)

      print('Meta-validation pre-inner-loop train loss: %.5f, meta-validation post-inner-loop test loss: %.5f' % (result[2], result[3][-1]))
      # print('Meta-validation pre-inner-loop train accuracy: %.5f, meta-validation post-inner-loop test accuracy: %.5f' % (result[-2], result[-1][-1]))

  model_file = logdir + '/' + exp_string +  '/model' + str(itr)
  print("Saving to ", model_file)
  model.save_weights(model_file)

# calculated for omniglot
NUM_META_TEST_POINTS = 600

def meta_test_fn(model, data_generator, n_way=5, meta_batch_size=25, k_shot=1,
              num_inner_updates=1):
  
  num_classes = data_generator.num_classes

  np.random.seed(1)
  random.seed(1)

  # meta_test_accuracies = []
  meta_test_losses = []

  for _ in range(NUM_META_TEST_POINTS):
    # sample a batch of test data and partition it into
    # the support/training set (input_tr, label_tr) and the query/test set (input_ts, label_ts)
    # NOTE: The code assumes that the support and query sets have the same number of examples.
    batch = data_generator.sample_batch("meta_test", meta_batch_size, shuffle=False, swap=False)
    #batch[0].shape(images) = (meta_batch_size, num_classes, k_shot*2, 784)
    #batch[1].shape(labels) = (meta_batch_size, num_classes, k_shot*2, num_classes)
    input_tr, input_ts = tf.split(batch[0], num_or_size_splits=2, axis=2)
    input_tr = tf.reshape(input_tr, [meta_batch_size, num_classes*k_shot, 784])
    input_ts = tf.reshape(input_ts, [meta_batch_size, num_classes*k_shot, 784])
    label_tr, label_ts = tf.split(batch[1], num_or_size_splits=2, axis=2)
    label_tr = tf.reshape(label_tr, [meta_batch_size, num_classes*k_shot, 784])
    label_ts = tf.reshape(label_ts, [meta_batch_size, num_classes*k_shot, 784])
    inp = (input_tr, input_ts, label_tr, label_ts)
    result = outer_eval_step(inp, model, meta_batch_size=meta_batch_size, num_inner_updates=num_inner_updates)

    # meta_test_accuracies.append(result[-1][-1])
    meta_test_losses.append(result[3][-1])

  # meta_test_accuracies = np.array(meta_test_accuracies)
  # means = np.mean(meta_test_accuracies)
  # stds = np.std(meta_test_accuracies)
  meta_test_losses = np.array(meta_test_losses)
  means = np.mean(meta_test_losses)
  stds = np.std(meta_test_losses)
  ci95 = 1.96*stds/np.sqrt(NUM_META_TEST_POINTS)

  # print('Mean meta-test accuracy/loss, stddev, and confidence intervals')
  print((means, stds, ci95))


def run_maml(n_way=5, k_shot=1, meta_batch_size=25, meta_lr=0.001,
             inner_update_lr=0.4, dim_hidden=32, num_inner_updates=1,
             learn_inner_update_lr=False,
             resume=False, resume_itr=0, log=True, logdir='/tmp/data',
             data_path='./omniglot_resized',meta_train=True,
             meta_train_iterations=15000, meta_train_k_shot=-1,
             meta_train_inner_update_lr=-1):


  # call data_generator and get data with k_shot*2 samples per class
  data_generator = DataGenerator(n_way, k_shot*2, n_way, k_shot*2, config={'data_folder': data_path})

  # set up MAML model
  dim_output = data_generator.dim_output
  dim_input = data_generator.dim_input
  model = MAML(dim_input,
              dim_output,
              num_inner_updates=num_inner_updates,
              inner_update_lr=inner_update_lr,
              k_shot=k_shot,
              dim_hidden=dim_hidden,
              learn_inner_update_lr=learn_inner_update_lr)

  if meta_train_k_shot == -1:
    meta_train_k_shot = k_shot
  if meta_train_inner_update_lr == -1:
    meta_train_inner_update_lr = inner_update_lr

  exp_string = 'cls_'+str(n_way)+'.mbs_'+str(meta_batch_size) + '.k_shot_' + str(meta_train_k_shot) + '.inner_numstep_' + str(num_inner_updates) + '.inner_updatelr_' + str(meta_train_inner_update_lr) + '.learn_inner_update_lr_' + str(learn_inner_update_lr)


  if meta_train:
    meta_train_fn(model, exp_string, data_generator,
                  n_way, meta_train_iterations, meta_batch_size, log, logdir,
                  k_shot, num_inner_updates, meta_lr)
  else:
    meta_batch_size = 1

    model_file = tf.train.latest_checkpoint(logdir + '/' + exp_string)
    print("Restoring model weights from ", model_file)
    model.load_weights(model_file)

    meta_test_fn(model, data_generator, n_way, meta_batch_size, k_shot, num_inner_updates)
  

In [202]:
run_maml(n_way=5, k_shot=1, inner_update_lr=0.4, num_inner_updates=1, dim_hidden=300)

Iteration 10: pre-inner-loop train loss: 0.16935, post-inner-loop test loss: 0.16920


KeyboardInterrupt: ignored